In [1]:
import os
path = "D:\projects\zimpleton\\backend\\app"
os.chdir(path)

In [3]:
os.getcwd()

'D:\\projects\\zimpleton\\backend\\app'

In [5]:
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import columns
from cassandra.cqlengine import connection
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine.query import BatchQuery

from lib.models import *
from lib.connections import set_datastax_connection,get_redis_connection
from lib.helpers import *
from lib.utils import *

In [2]:
# connection.execute("truncate table user_relation")
connection.execute("drop table pins")

# connection.execute("truncate table user_info")
# connection.execute("truncate table user_credentials")

NameError: name 'connection' is not defined

In [ ]:
r = get_redis_connection()
# r.flushdb()

In [6]:
query = "SELECT keyspace_name FROM system_schema.keyspaces;"
ckeyspaces = connection.execute(query)
ckeyspaces = [keyspace["keyspace_name"] for keyspace in ckeyspaces]
ckeyspaces

['xkeyspace',
 'unispace',
 'system_auth',
 'system_schema',
 'system',
 'datastax_sla',
 'pinspace',
 'system_traces',
 'data_endpoint_auth']

In [7]:
credvalues = UserCredentials.objects.all()
list(credvalues)

[UserCredentials(email='pi@gmail.com', hashed_password='$2b$12$pEKVpiZInfx4yeTb0VzwMea3YvLSUdSN3WeI59GbW5OTujyIfikY2'),
 UserCredentials(email='ip@gmail.com', hashed_password='$2b$12$DBqMI9IwqvvuS6Z7RZfLeOasYjJereWQEDPKVdizsEjw6hA0eVW8O'),
 UserCredentials(email='james@gmail.com', hashed_password='$2b$12$LAsg.vvBVcTJo2VI1BUMh.BeFuS0h3NNxlR/zaoUqq0sNUI74Leoy')]

In [8]:
infovalues = UserInfo.objects.all()
list(infovalues)

[UserInfo(email='pi@gmail.com', name='pi@gmail.com'),
 UserInfo(email='ip@gmail.com', name='ip@gmail.com'),
 UserInfo(email='james@gmail.com', name='james')]

In [148]:
import uuid
# class Pins(Model):
#     id = columns.Text(primary_key=True)
#     email = columns.Text()
#     pintimestamp = columns.DateTime()

#     title = columns.Text()
#     image = columns.Blob()

# class Pins(Model):
#     id = columns.UUID(primary_key=True, default=uuid.uuid4)
#     email = columns.Text(partition_key = True)

#     title = columns.Text()
#     content = columns.Text()

class Tins(Model):
    email = columns.Text(primary_key = True, partition_key = True)
    rank = columns.Integer(primary_key=True, clustering_order="DESC")
    
    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    title = columns.Text()
    content = columns.Text()

class Posts(Model):
    email = columns.Text(primary_key = True, partition_key = True)
    xtimestamp = columns.DateTime(primary_key=True,clustering_order="DESC",default=datetime.now)

    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    title = columns.Text()
    content = columns.Text()

class Pins(Model):
    email = columns.Text(primary_key = True, partition_key = True)
    xtimestamp = columns.DateTime(primary_key=True,clustering_order="DESC",default=datetime.now)

    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    title = columns.Text()
    image = columns.Blob()

connection.execute("drop table tins")
connection.execute("drop table posts")
connection.execute("drop table pins")

sync_table(Tins)
sync_table(Posts)
sync_table(Pins)

d:\projects\zimpleton\backend\menvi\Lib\site-packages\cassandra\cqlengine\management.py:544: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [131]:
# TINS
import uuid
import random
import string

def add_random_tins():
    # a list of example emails
    emails = ['user1@example.com', 'user2@example.com', 'user3@example.com']

    # a list of example titles
    titles = ['First Pin', 'Second Pin', 'Third Pin', 'Fourth Pin', 'Fifth Pin']

    # generate random content
    content = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.punctuation, k=20))

    # add 10 random pins
    for _ in range(10):
        # select a random email and title
        email = random.choice(emails)
        title = random.choice(titles)

        # generate a random rank between 1 and 100
        rank = random.randint(1, 100)

        # create a new pin
        pin = Pins(
            id=uuid.uuid4(),
            email=email,
            rank=rank,
            title=title,
            content=content
        )

        pin.save()
add_random_tins()

In [128]:
# POSTS
def create_random_post():
    emails = ['user1@example.com', 'user2@example.com', 'user3@example.com']

    email = random.choice(emails)

    post = Posts(
        email=email,
        title='Random Post ' + str(uuid.uuid4()),
        content='This is a random post with ID ' + str(uuid.uuid4()),
    )

    post.save()

for _ in range(10) : create_random_post()


In [146]:
def update_post(post_id, new_title, new_content):
    try:
        post = Posts.objects.allow_filtering().get(id=post_id)
        post.title = new_title
        post.content = new_content
        post.save()
        return True
    except Posts.DoesNotExist:
        print(f"Post with id {post_id} does not exist.")
        return False
    except Exception as e:
        print(f"Validation Error: {e}")
        return False

post_id = "21e8ec2a-8232-48b2-95e3-7fe6813a8d41"
update_post(post_id= post_id, new_title = "edited title here", new_content = "edited content here")

True

In [9]:
Posts.objects.allow_filtering().get(id= post_id)

NameError: name 'Posts' is not defined

In [144]:
post_id = "0672ef9b-c64c-4ac4-8b22-065ad8625499"
Posts.objects.allow_filtering().get(id = post_id).delete()

In [142]:
email = 'user3@example.com'

# values = Tins.objects(email=email)
# values = Pins.objects(email=email)
values = Posts.objects(email=email)

list(values)

# 0672ef9b-c64c-4ac4-8b22-065ad8625499

[Posts(email='user3@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 21, 17, 379000), id=UUID('21e8ec2a-8232-48b2-95e3-7fe6813a8d41'), title='edited title here', content='edited content here'),
 Posts(email='user3@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 21, 16, 398000), id=UUID('a91a6ac7-0423-4dc2-b251-c41f75fd7d03'), title='Random Post e72eb576-a9b7-4bca-bd6e-80e7ac0f338c', content='This is a random post with ID 91874a20-6734-4de6-87ef-b7a351844af4'),
 Posts(email='user3@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 21, 16, 146000), id=UUID('bd43022a-a418-48a5-90c2-57fb7eb044f0'), title='Random Post 137aa99c-e693-4e6f-9da4-cc4c3559968a', content='This is a random post with ID 79b58f19-92bb-4e96-9b06-d21e8195b126'),
 Posts(email='user3@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 21, 15, 896000), id=UUID('0672ef9b-c64c-4ac4-8b22-065ad8625499'), title='Random Post 6b3993f4-3695-4c02-b350-2a9453e4110f', content='This is a random

In [127]:
# values = Tins.objects().all()
# values = Pins.objects().all()
values = Posts.objects().all()

list(values)

[Posts(email='user2@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 20, 37, 921000), id=UUID('14adef11-b5fb-4d4d-8a91-b86012d3dac1'), title='Random Post 7a67864b-17a8-426b-a1d2-e22186ca4804', content='This is a random post with ID d98ab908-94c8-43c3-b78f-4f781149f782'),
 Posts(email='user2@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 20, 37, 179000), id=UUID('bc8984bd-915e-4377-b29b-cb2906e6da2c'), title='Random Post 1415ef74-a608-4560-9f2c-32efc1b0f6fe', content='This is a random post with ID 473d91a5-dfb2-4bc5-b924-346cbd970e23'),
 Posts(email='user2@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 20, 35, 945000), id=UUID('ce82c66b-380e-42d7-bd05-8045bbecf13a'), title='Random Post b5e98fac-826f-45f0-91df-73a1d95adb86', content='This is a random post with ID 67067f9e-912b-4066-bf95-aa5ea8cb9bdc'),
 Posts(email='user2@example.com', xtimestamp=datetime.datetime(2024, 4, 23, 12, 20, 34, 139000), id=UUID('b406c3a0-2ca2-4fff-9b5d-277b2ee83345'), title

In [168]:
pin = Pins.objects.all().get()
str(pin.id)

did = 'a1c1cefe-f0df-4342-b89e-8d039f53730c'

In [172]:
pin.image

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x04\x04\x04\x04\x04\x06\x05\x05\x05\x05\x06\t\x06\x07\x06\x07\x06\t\x0e\x08\n\x08\x08\n\x08\x0e\x0c\x0f\x0c\x0b\x0c\x0f\x0c\x16\x11\x0f\x0f\x11\x16\x19\x15\x14\x15\x19\x1e\x1b\x1b\x1e&$&22C\x01\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x04\x04\x04\x04\x04\x06\x05\x05\x05\x05\x06\t\x06\x07\x06\x07\x06\t\x0e\x08\n\x08\x08\n\x08\x0e\x0c\x0f\x0c\x0b\x0c\x0f\x0c\x16\x11\x0f\x0f\x11\x16\x19\x15\x14\x15\x19\x1e\x1b\x1b\x1e&$&22C\xff\xc2\x00\x11\x08\x02\x80\x03;\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1e\x00\x00\x03\x00\x03\x00\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\xff\xda\x00\x08\x01\x01\x00\x00\x00\x00\xfb\xb3\x90\xdd66\xc2\x906\x00\xe90\x1b\x01\nT\xea\xf2>=\xfc\xe0\xc7\xa9\x00}C\x83\xd2X\xf3\xe4\xbfr\xfe\xb1l\xed\xe4\xa4:\x00\x06\xa4m\x88\x18!\xb2\x86\xc6\x00\xd7!\xb6\xdbm\xb1\x88m\xa6\x98\xc1\x8c\x00&Tkr~=\x